In [31]:
############尚未優化前的fingerprint###########
##########組合: (NN,CA)/(NN,center)/(CA,center)########
import numpy as np
import math
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import os
def Dis_self(a_x,a_y,a_z,b_x,b_y,b_z):####算自己原子內的距離
  ab_x=0
  ab_y=0
  ab_z=0
  ab_x=a_x-b_x
  ab_y=a_y-b_y
  ab_z=a_z-b_z
  ab_dis=math.pow((ab_x*ab_x+ab_y*ab_y+ab_z*ab_z),0.5)
  return ab_dis

In [32]:
import zipfile
with zipfile.ZipFile("./known_property_single_centroid_dataset.zip", "r") as zip_ref:
    zip_ref.extractall("./known_property_single_centroid_dataset")####要自己創一個放壓縮後data的資料夾########

In [33]:
## 導入pandas套件
import pandas as pd

excel_data_df = pd.read_excel('./features_only.xlsx')
total_protein=excel_data_df['Protein pdb'].tolist()
feature=excel_data_df['Volume'].tolist()
#feature2=excel_data_df['total seq num'].tolist()
#feature3=excel_data_df['total seq num'].tolist()
## 顯示數據
print(total_protein)
Negative=["UBIQUITIN","SUMO","1XHD","1DLY","1T3Y","4BPY","5LDA_A","5LDA_B","3QF2","5CWK",
        "1QFT","5OLL","6VAN","5VEG","5M2P","6R3M","4QBB","2OOB_A","2OOB_B","2CE0","4RZ9","6TRK",
        "1DUS","5FA8","4HLB","5Z0W_A","5Z0W_B","6XAZ","4J7Z","5J0C","6RYG","1JL1","2XNR"]
Positive=["DMP19_18","HI1450","GAM","Arn_dimer_new","OCR_dimer","MFPA","NuiA","AcrF10","DinI","p56","AcrF2","AcrIIA4",
       "UGI","ICP11","Abba","SAUGI","CarSnew","ArdA","DMP12"]
protein_pdb=[]
for i in range(len(total_protein)):
  if total_protein[i] in Negative:
    protein_pdb.append(total_protein[i])
  if total_protein[i] in Positive:
    protein_pdb.append(total_protein[i])
print(protein_pdb)

['3QF2', 'SAUGI', '6TRK', 'p56', '1DLY', '5VEG', 'AcrF10', '1QFT', 'NuiA', '2OOB_B', '5FA8', '4J7Z', '5OLL', 'UGI', 'DMP12', 'ArdA', 'DMP19_18', 'AcrIIA4', 'Abba', '1DUS', '1T3Y', '6VAN', 'CarSnew', '2OOB_A', 'OCR_dimer', '2XNR', '1XHD', '5Z0W_B', 'MFPA', '4HLB', 'HI1450', '5Z0W_A', '4QBB', 'SUMO', '5LDA_B', '6XAZ', '5M2P', '5J0C', 'AcrF2', '5LDA_A', '6RYG', 'ICP11', 'GAM', '5CWK', 'Arn_dimer_new', '1JL1', '2CE0', '6R3M', 'DinI', 'UBIQUITIN', '4RZ9', '4BPY']
['3QF2', 'SAUGI', '6TRK', 'p56', '1DLY', '5VEG', 'AcrF10', '1QFT', 'NuiA', '2OOB_B', '5FA8', '4J7Z', '5OLL', 'UGI', 'DMP12', 'ArdA', 'DMP19_18', 'AcrIIA4', 'Abba', '1DUS', '1T3Y', '6VAN', 'CarSnew', '2OOB_A', 'OCR_dimer', '2XNR', '1XHD', '5Z0W_B', 'MFPA', '4HLB', 'HI1450', '5Z0W_A', '4QBB', 'SUMO', '5LDA_B', '6XAZ', '5M2P', '5J0C', 'AcrF2', '5LDA_A', '6RYG', 'ICP11', 'GAM', '5CWK', 'Arn_dimer_new', '1JL1', '2CE0', '6R3M', 'DinI', 'UBIQUITIN', '4RZ9', '4BPY']


In [34]:
################NN非累加##############
from sklearn.metrics import mean_squared_error
class Protein:
    def __init__(self, ATOM, serial, name, resName, chainID, resSeq, x, y, z, temp1, temp2, temp3,temp4):
        self.ATOM = ATOM  # 記錄類型
        self.serial = serial  # 原子序號
        self.name = name  # 原子名稱
        self.resName = resName  # 殘基名稱
        self.chainID = chainID  # Chain identifier 鏈標記號
        self.resSeq = resSeq  # Residue sequence number 殘基序列號
        self.x = x  # 直角X坐標
        self.y = y  # 直角y坐標
        self.z = z  # 直角z坐標
        self.temp1 = temp1
        self.temp2 = temp2
        self.temp3 = temp3
        self.temp4 = temp4


mode1 = ['ASP','GLU']
mode3 = ['OE1AGLU','OE2AGLU','OD1AASP','OD2AASP']
mode2 = ['OE1','OE2','OD1','OD2']





target_x=[]
target_y=[]
target_z=[]
count_tar=0
count_obj=0

NxN_noaccum_fingerprint=[]

####存x/y/z座標########
for j in range(len(protein_pdb)):

  ##########atom_dis################記錄在範圍內的園子個數(索引0代表距離在0~1內的原子個數/索引1代表距離在1~2內的原子個數)###
  atom_num=[0]*148
  ####紀錄OE1/OE2及OD1/OD2的平均座標#########
  avg_negetive_atom_x = []
  avg_negetive_atom_y = []
  avg_negetive_atom_z = []
  inputPath='./known_property_single_centroid_dataset/'+protein_pdb[j]+'.pdb'
  print("Protein Name: ",protein_pdb[j])

  x_tar=[]
  y_tar=[]
  z_tar=[]
  with open(inputPath) as f:
    print(f)
    lines = f.readlines()
  data = []
  results = []
  for line in lines:
  #print(line)
    x = line.split()
    if x[0] == 'END':
      break
    if x[0] != 'ATOM':
      continue
    if x[0] == 'TER':
      continue

    if len(x)<11:
      continue
    if len(x)==12:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],"Empty", x[11])
    if len(x)==11:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],"Empty","Empty")
    if len(x)==13:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],x[11],x[12])

    data.append(protein)

    if ((protein.resName in mode1) ) and (protein.name in mode2):
      split_line=line.split( )
      x_tar.append(split_line[6])
      y_tar.append(split_line[7])
      z_tar.append(split_line[8])

      results.append(line)
    if (protein.name in mode3):
      if protein.temp3=="Empty":
        #print("protein.name=",protein.name)
        split_line=line.split( )
        x_tar.append(split_line[5])
        y_tar.append(split_line[6])
        z_tar.append(split_line[7])
        results.append(line)
  num_tar=len(results)
  print("The number of ASP/GLU in target pdb =", num_tar)

  #####convert list to numpy array###########
  x_np_tar = np.array(x_tar,dtype=np.float32)
  y_np_tar = np.array(y_tar,dtype=np.float32)
  z_np_tar = np.array(z_tar,dtype=np.float32)



  for i in range(0,len(x_np_tar),2):
    avg_negetive_atom_x.append((x_np_tar[i]+x_np_tar[i+1])/2)
    avg_negetive_atom_y.append((y_np_tar[i]+y_np_tar[i+1])/2)
    avg_negetive_atom_z.append((z_np_tar[i]+z_np_tar[i+1])/2)

  for e in range(len(avg_negetive_atom_x)):
    for k in range(e+1,len(avg_negetive_atom_x)):
      dis=Dis_self(avg_negetive_atom_x[e],avg_negetive_atom_y[e],avg_negetive_atom_z[e],avg_negetive_atom_x[k],avg_negetive_atom_y[k],avg_negetive_atom_z[k])
      for k in range(0,len(atom_num)):
        if dis>=k+3 and dis<k+4:
          atom_num[k]=atom_num[k]+1
          break

  ##########after normalize#############
  for e in range(len(atom_num)):
    atom_num[e]=atom_num[e]*e/feature[j]

  print(atom_num)
  NxN_noaccum_fingerprint.append(atom_num)

Protein Name:  3QF2
<_io.TextIOWrapper name='./known_property_single_centroid_dataset/3QF2.pdb' mode='r' encoding='UTF-8'>
The number of ASP/GLU in target pdb = 34
[0.0, 0.0, 0.016444640948639084, 0.024666961422958624, 0.13155712758911267, 0.08222320474319542, 0.14800176853775174, 0.34533745992142073, 0.32889281897278166, 0.22200265280662762, 0.16444640948639083, 0.7235642017401196, 0.8880106112265105, 0.5344508308307702, 0.9208998931237886, 0.6166740355739656, 1.3155712758911267, 0.8386766883805932, 0.8880106112265105, 0.7811204450603564, 1.4800176853775173, 1.7266872996071037, 1.4471284034802392, 1.3237935963654461, 0.789342765534676, 0.6166740355739656, 0.4275606646646161, 0.22200265280662762, 0.23022497328094715, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [35]:
##############算NXother 挑CA當other 非累加######################
from sklearn.metrics import mean_squared_error
class Protein:
    def __init__(self, ATOM, serial, name, resName, chainID, resSeq, x, y, z, temp1, temp2, temp3,temp4):
        self.ATOM = ATOM  # 記錄類型
        self.serial = serial  # 原子序號
        self.name = name  # 原子名稱
        self.resName = resName  # 殘基名稱
        self.chainID = chainID  # Chain identifier 鏈標記號
        self.resSeq = resSeq  # Residue sequence number 殘基序列號
        self.x = x  # 直角X坐標
        self.y = y  # 直角y坐標
        self.z = z  # 直角z坐標
        self.temp1 = temp1
        self.temp2 = temp2
        self.temp3 = temp3
        self.temp4 = temp4


mode1 = ['ASP','GLU']
mode3 = ['OE1AGLU','OE2AGLU','OD1AASP','OD2AASP']
mode2 = ['OE1','OE2','OD1','OD2']
mode4 = ['CA']

#print(arr_maximum_bipartite_matching)
target_x=[]
target_y=[]
target_z=[]
count_tar=0
count_obj=0
Nxother_noaccum_fingerprint=[]
#NxN_noaccum_fingerprint=[]
#NxN_noaccum_fingerprint=[]
####存x/y/z座標########
for j in range(len(protein_pdb)):
  atom_num=[0]*151
  ####紀錄OE1/OE2及OD1/OD2的平均座標#########
  avg_negetive_atom_x = []
  avg_negetive_atom_y = []
  avg_negetive_atom_z = []
  inputPath='./known_property_single_centroid_dataset/'+protein_pdb[j]+'.pdb'

  x_tar=[]
  y_tar=[]
  z_tar=[]
  CA_x=[]
  CA_y=[]
  CA_z=[]
  with open(inputPath) as f:
    print(f)
    lines = f.readlines()
  data = []
  results = []
  for line in lines:
  #print(line)
    x = line.split()
    if x[0] == 'END':
      break
    if x[0] != 'ATOM':
      continue
    if x[0] == 'TER':
      continue

    if len(x)<11:
      continue
    if len(x)==12:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],"Empty", x[11])
    if len(x)==11:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],"Empty","Empty")
    if len(x)==13:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],x[11],x[12])

    data.append(protein)
    if (protein.name in mode4):
      split_line=line.split( )
      CA_x.append(float(split_line[6]))
      CA_y.append(float(split_line[7]))
      CA_z.append(float(split_line[8]))
    if ((protein.resName in mode1) ) and (protein.name in mode2):
      split_line=line.split( )
      x_tar.append(split_line[6])
      y_tar.append(split_line[7])
      z_tar.append(split_line[8])

      results.append(line)
    if (protein.name in mode3):
      if protein.temp3=="Empty":
        #print("protein.name=",protein.name)
        split_line=line.split( )
        x_tar.append(split_line[5])
        y_tar.append(split_line[6])
        z_tar.append(split_line[7])
        results.append(line)
  num_tar=len(results)
  print("The number of ASP/GLU in target pdb =", num_tar)
  #print("Target X/Y/Z")
  #####convert list to numpy array###########
  #print(x_tar)
  x_np_tar = np.array(x_tar,dtype=np.float32)
  y_np_tar = np.array(y_tar,dtype=np.float32)
  z_np_tar = np.array(z_tar,dtype=np.float32)



  for i in range(0,len(x_np_tar),2):
    avg_negetive_atom_x.append((x_np_tar[i]+x_np_tar[i+1])/2)
    avg_negetive_atom_y.append((y_np_tar[i]+y_np_tar[i+1])/2)
    avg_negetive_atom_z.append((z_np_tar[i]+z_np_tar[i+1])/2)

  for e in range(len(avg_negetive_atom_x)):
    for k in range(len(CA_x)):
      dis=Dis_self(avg_negetive_atom_x[e],avg_negetive_atom_y[e],avg_negetive_atom_z[e],CA_x[k],CA_y[k],CA_z[k])
      for n in range(0,len(atom_num)):
        if dis>=n and dis<n+1:
          atom_num[n]=atom_num[n]+1
          break
  ##########after normalize#############
  for e in range(len(atom_num)):
    atom_num[e]=atom_num[e]/feature[j]
  print(atom_num)
  #NxN_noaccum_fingerprint.append(atom_num)
  Nxother_noaccum_fingerprint.append(atom_num)
  #Nxother_noaccum_fingerprint.append(atom_num)

<_io.TextIOWrapper name='./known_property_single_centroid_dataset/3QF2.pdb' mode='r' encoding='UTF-8'>
The number of ASP/GLU in target pdb = 34
[0.0, 0.0, 0.008222320474319542, 0.08222320474319542, 0.1808910504350299, 0.16444640948639083, 0.2384472937552667, 0.3371151394471012, 0.394671382767338, 0.34533745992142073, 0.4851169079848529, 0.34533745992142073, 0.5344508308307702, 0.6413409969969243, 0.6577856379455633, 0.7811204450603564, 0.6495633174712437, 0.7482311631630783, 0.8797882907521909, 0.7153418812658001, 0.6166740355739656, 0.805787406483315, 0.5591177922537288, 0.6248963560482851, 0.4851169079848529, 0.4193383441902966, 0.31244817802414254, 0.26311425517822534, 0.11511248664047358, 0.08222320474319542, 0.06577856379455634, 0.024666961422958624, 0.016444640948639084, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [36]:
########Center###########

from sklearn.metrics import mean_squared_error
class Protein:
    def __init__(self, ATOM, serial, name, resName, chainID, resSeq, x, y, z, temp1, temp2, temp3,temp4):
        self.ATOM = ATOM  # 記錄類型
        self.serial = serial  # 原子序號
        self.name = name  # 原子名稱
        self.resName = resName  # 殘基名稱
        self.chainID = chainID  # Chain identifier 鏈標記號
        self.resSeq = resSeq  # Residue sequence number 殘基序列號
        self.x = x  # 直角X坐標
        self.y = y  # 直角y坐標
        self.z = z  # 直角z坐標
        self.temp1 = temp1
        self.temp2 = temp2
        self.temp3 = temp3
        self.temp4 = temp4


mode1 = ['ASP','GLU']
mode3 = ['OE1AGLU','OE2AGLU','OD1AASP','OD2AASP']
mode2 = ['OE1','OE2','OD1','OD2']
mode4 = ['CA']

#print(arr_maximum_bipartite_matching)
target_x=[]
target_y=[]
target_z=[]
count_tar=0
count_obj=0

center_noaccum_fingerprint=[]

####存x/y/z座標########
for j in range(len(protein_pdb)):
  atom_num=[0]*151
  ####紀錄OE1/OE2及OD1/OD2的平均座標#########
  avg_negetive_atom_x = []
  avg_negetive_atom_y = []
  avg_negetive_atom_z = []
  inputPath='./known_property_single_centroid_dataset/'+protein_pdb[j]+'.pdb'

  x_tar=[]
  y_tar=[]
  z_tar=[]
  CA_x=[]
  CA_y=[]
  CA_z=[]
  with open(inputPath) as f:
    print(f)
    lines = f.readlines()
  data = []
  results = []
  for line in lines:
  #print(line)
    x = line.split()
    if x[0] == 'END':
      break
    if x[0] != 'ATOM':
      continue
    if x[0] == 'TER':
      continue

    if len(x)<11:
      continue
    if len(x)==12:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],"Empty", x[11])
    if len(x)==11:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],"Empty","Empty")
    if len(x)==13:
      protein = Protein(x[0], x[1], x[2], x[3], x[4],
                x[5], x[6], x[7], x[8], x[9], x[10],x[11],x[12])

    data.append(protein)
    if ((protein.resName in mode1) ) and (protein.name in mode4):
      split_line=line.split( )
      CA_x.append(float(split_line[6]))
      CA_y.append(float(split_line[7]))
      CA_z.append(float(split_line[8]))
    if ((protein.resName in mode1) ) and (protein.name in mode2):
      split_line=line.split( )
      x_tar.append(split_line[6])
      y_tar.append(split_line[7])
      z_tar.append(split_line[8])

      results.append(line)
    if (protein.name in mode3):
      if protein.temp3=="Empty":
        #print("protein.name=",protein.name)
        split_line=line.split( )
        x_tar.append(split_line[5])
        y_tar.append(split_line[6])
        z_tar.append(split_line[7])
        results.append(line)
  num_tar=len(results)
  print("The number of ASP/GLU in target pdb =", num_tar)
  #print("Target X/Y/Z")
  #####convert list to numpy array###########
  #print(x_tar)
  x_np_tar = np.array(x_tar,dtype=np.float32)
  y_np_tar = np.array(y_tar,dtype=np.float32)
  z_np_tar = np.array(z_tar,dtype=np.float32)



  for i in range(0,len(x_np_tar),2):
    avg_negetive_atom_x.append((x_np_tar[i]+x_np_tar[i+1])/2)
    avg_negetive_atom_y.append((y_np_tar[i]+y_np_tar[i+1])/2)
    avg_negetive_atom_z.append((z_np_tar[i]+z_np_tar[i+1])/2)

  for e in range(len(avg_negetive_atom_x)):
    dis=Dis_self(avg_negetive_atom_x[e],avg_negetive_atom_y[e],avg_negetive_atom_z[e],0,0,0)
    for n in range(0,len(atom_num)):
      if dis>=n and dis<n+1:
        atom_num[n]=atom_num[n]+1
        break
  ##########after normalize#############
  for e in range(len(atom_num)):
    atom_num[e]=atom_num[e]*e/feature[j]
  print(atom_num)
  center_noaccum_fingerprint.append(atom_num)

<_io.TextIOWrapper name='./known_property_single_centroid_dataset/3QF2.pdb' mode='r' encoding='UTF-8'>
The number of ASP/GLU in target pdb = 34
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4933392284591725, 0.32067049849846213, 0.5755624332023679, 0.12333480711479312, 0.26311425517822534, 0.1397794480634322, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
<_io.Te

In [37]:
##############將經過normalize的fingerprint寫入excel###############
import os
os.chdir('./')  # Colab 換路徑使用
import openpyxl
wb = openpyxl.load_workbook('empty_file.xlsx', data_only=True)
s1 = wb['工作表1']            # 開啟工作表 1

s1['A1'].value = 'Protein pdb'
for i in range(len(protein_pdb)):
  s1.cell(2+i,1).value=protein_pdb[i]

local_atom_length=148
global_atom_length=151
for i in range(local_atom_length):
  s1.cell(1,i+2).value="N"+str(i+3)+"N"

for i in range(global_atom_length):
  s1.cell(1,i+2+local_atom_length).value="N"+str(i)+"Other"
for i in range(global_atom_length):
  s1.cell(1,i+2+global_atom_length+local_atom_length).value="N"+str(i)+"Centre"
#  s1.cell(1,i+2+local_atom_length).value="N"+str(i)+"Center"

######### NN ##############
for i in range(len(NxN_noaccum_fingerprint)):
  for j in range(len(NxN_noaccum_fingerprint[i])):
    s1.cell(2+i,2+j).value=NxN_noaccum_fingerprint[i][j]

########## CA ##############

for i in range(len(Nxother_noaccum_fingerprint)):
  for j in range(len(Nxother_noaccum_fingerprint[i])):
    s1.cell(2+i,2+j+local_atom_length).value=Nxother_noaccum_fingerprint[i][j]


########## center ###########
for i in range(len(center_noaccum_fingerprint)):
  for j in range(len(center_noaccum_fingerprint[i])):
    s1.cell(2+i,2+j+local_atom_length+global_atom_length).value=center_noaccum_fingerprint[i][j]


wb.save('known_property_single_centroid_dataset_Combinedfingerprint.xlsx')